In [1]:
import pandas as pd
import tensorflow as tf

In [2]:
df = pd.read_csv("main_database_small_diffs_draws_or_not.csv")

In [3]:
df.head(2)

,season,target,PosDiff,GSPGDiff,GCPGDiff,CSPGDiff,PtsPGDiff,SPGDiff,STPGDiff,S%Diff,PTSPG3Diff,PTSPG5Diff,PTSPG7Diff,GD3Diff,GD5Diff,GD7Diff,PosLSDiff,SpentDiff,IncomeDiff,NetDiff
0,9,0,-9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.65,-0.841,-1.491
1,9,0,-9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,-12.42,14.650,27.070


In [4]:
df['target'].value_counts()

0    3453
1    1264
Name: target, dtype: int64

In [5]:
3453/1264

2.7318037974683542

In [6]:
from sklearn.model_selection import StratifiedShuffleSplit

def get_train_test_split(data):
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    for train_index, test_index in split.split(data, data["target"]):
        data_train = data.loc[train_index]
        data_test = data.loc[test_index]
    return data_train, data_test

train,test = get_train_test_split(df)

X_train = train.drop('target',axis=1)
y_train = train['target']
X_test = test.drop('target',axis=1)
y_test = test['target']

In [7]:
X_train.head(2)

,season,PosDiff,GSPGDiff,GCPGDiff,CSPGDiff,PtsPGDiff,SPGDiff,STPGDiff,S%Diff,PTSPG3Diff,PTSPG5Diff,PTSPG7Diff,GD3Diff,GD5Diff,GD7Diff,PosLSDiff,SpentDiff,IncomeDiff,NetDiff
3242,20,-2.0,0.07,-0.08,0.11,0.07,3.593,2.111,-0.016,-2.333,-1.4,-1.143,-2.000,-0.8,-0.857,-1.0,53.39,-3.51,-56.9
521,10,1.0,-0.18,-0.18,0.06,0.00,0.294,0.117,0.000,-1.333,-0.6,-0.429,-2.333,-1.4,-1.143,-1.0,-5.27,-11.77,-6.5


In [8]:
X_train_norm = tf.keras.utils.normalize(X_train.values,axis=1)
X_test_norm = tf.keras.utils.normalize(X_test.values,axis=1)
X_train_norm[0]

array([ 2.47368308e-01, -2.47368308e-02,  8.65789077e-04, -9.89473231e-04,
        1.36052569e-03,  8.65789077e-04,  4.44397165e-02,  2.61097249e-02,
       -1.97894646e-04, -2.88555131e-02, -1.73157815e-02, -1.41370988e-02,
       -2.47368308e-02, -9.89473231e-03, -1.05997320e-02, -1.23684154e-02,
        6.60349697e-01, -4.34131380e-02, -7.03762835e-01])

In [9]:
model = tf.keras.models.Sequential()
#model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(12, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(12, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(2, activation = tf.nn.softmax))

In [10]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

class_weight = {0: 1,
                1: 2.732}

In [11]:
model.fit(X_train_norm, y_train.values, epochs=20, validation_data=(X_test_norm,y_test), class_weight=class_weight)

Train on 3773 samples, validate on 944 samples
Epoch 1/20
3773/3773 [==============================] - 1s 391us/step - loss: 0.6938 - acc: 0.6268 - val_loss: 0.6813 - val_acc: 0.6907
Epoch 2/20
3773/3773 [==============================] - 1s 173us/step - loss: 0.6918 - acc: 0.6581 - val_loss: 0.6841 - val_acc: 0.6356
Epoch 3/20
3773/3773 [==============================] - 1s 261us/step - loss: 0.6910 - acc: 0.6255 - val_loss: 0.6810 - val_acc: 0.6335
Epoch 4/20
3773/3773 [==============================] - 0s 120us/step - loss: 0.6903 - acc: 0.6300 - val_loss: 0.6796 - val_acc: 0.6271
Epoch 5/20
3773/3773 [==============================] - 1s 135us/step - loss: 0.6893 - acc: 0.6311 - val_loss: 0.6795 - val_acc: 0.6091
Epoch 6/20
3773/3773 [==============================] - 1s 143us/step - loss: 0.6888 - acc: 0.6316 - val_loss: 0.6788 - val_acc: 0.6112
Epoch 7/20
3773/3773 [==============================] - 1s 172us/step - loss: 0.6884 - acc: 0.6157 - val_loss: 0.6806 - val_acc: 0.5985
E

In [12]:
train_pred = model.predict_classes(X_train_norm)
test_pred = model.predict_classes(X_test_norm)

import numpy as np

unique, counts = np.unique(train_pred, return_counts=True)
print("preds: " + str(dict(zip(unique, counts))))

unique, counts = np.unique(y_train.values, return_counts=True)
print("real: " + str(dict(zip(unique, counts))))

from sklearn.metrics import confusion_matrix as conf_matrix

print(conf_matrix(train_pred,y_train.values,labels=[0,1]))

preds: {0: 2396, 1: 1377}
real: {0: 2762, 1: 1011}
[[1843  553]
 [ 919  458]]


In [13]:
print(conf_matrix(test_pred,y_test.values))

[[429 163]
 [262  90]]


In [14]:
test_pred_probs = model.predict(X_test_norm)

In [15]:
test_pred_probs[0]

array([0.552376, 0.447624], dtype=float32)

In [ ]:
2184/1264 #1

In [ ]:
2184/1269 #0